# template から HTML と JS を生成する
## テスト用のデータを test-tsumugi にコピー


In [1]:
import os
from pathlib import Path
import json
import gzip
from collections import defaultdict

import pandas as pd

In [2]:
# Move up to top directory

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI-dev-branch/notebooks/notebools-web
/mnt/e/Research/TSUMUGI-dev-branch


# 古いファイルの消去


In [3]:
%%bash

rm -rf test-tsumugi/data
rm -rf test-tsumugi/app


In [4]:
%%bash

mkdir -p test-tsumugi/data/phenotype
mkdir -p test-tsumugi/data/genesymbol
mkdir -p test-tsumugi/app/phenotype
mkdir -p test-tsumugi/app/genesymbol
mkdir -p test-tsumugi/app/genelist/

cp -r TSUMUGI/css test-tsumugi/ # copy top's CSS
cp -r TSUMUGI/js test-tsumugi/ # copy top's JS

cp -r TSUMUGI/template/css test-tsumugi/app/ # copy app's CSS
cp -r TSUMUGI/template/js test-tsumugi/app/ # copy app's JS


## index.html の描画に必要なファイルを生成する


In [5]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    if not Path("data", "network", "mp_term_name", f"{mp_term}.json.gz").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    mp_terms[mp_term_name_space] = mp_term

In [6]:
print(list(mp_terms)[:3])
print(len(mp_terms))  # 515

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
517


In [7]:
json.dump(mp_terms, open("data/overlap/available_mp_terms.json", "w"), indent=2)
pd.DataFrame(mp_terms.keys()).to_csv(
    "data/overlap/available_mp_terms.txt", index=False, header=False
)

In [8]:
%%bash

cp data/overlap/available_mp_terms.json test-tsumugi/data/available_mp_terms.json
cp data/overlap/available_mp_terms.txt test-tsumugi/data/available_mp_terms.txt
cp data/overlap/available_gene_symbols.txt test-tsumugi/data/available_gene_symbols.txt
cp data/annotation/symbol_mgiid.json test-tsumugi/data/marker_symbol_accession_id.json


## index.html を生成


In [9]:
%%bash

grep -v "REMOVE_THIS_LINE" TSUMUGI/template/template_index.html > test-tsumugi/index.html

# appのHTMLとJSを生成


### binariyの表現型を抽出

In [10]:
path_files = Path("TSUMUGI", "data", "phenotype").glob("*.json.gz")
binary_phenotypes = []
for path_file in path_files:
    mp_term_name = path_file.name.replace(".json.gz", " ")
    with gzip.open(path_file, "rt") as f:
        json_data: list[dict] = json.load(f)

    count_node_color = defaultdict(int)
    for data in json_data:
        if "node_color" not in data["data"]:
            continue
        count_node_color[data["data"]["node_color"]] += 1
    if len(count_node_color) == 1 and 1 in count_node_color:
        binary_phenotypes.append(mp_term_name)

print(len(binary_phenotypes))
print(binary_phenotypes[:3])
Path("data", "annotation", "binary_phenotypes.txt").write_text("\n".join(binary_phenotypes))

154
['abnormal_abdominal_wall_morphology ', 'abnormal_adrenal_gland_morphology ', 'abnormal_allantois_morphology ']


3761

In [11]:
# %%bash

# ########################################
# # Phenotype
# ########################################

# find data/mp_term_name -type f |
#     sed "s|data/mp_term_name/||" |
#     sed "s|.csv$||" |
#     grep -e ^male_infertility -e ^increased_fasting -e "^prewe" | # <- ここで興味のあるphenotypeを選択
#     while read -r mp_term_name_underscore; do
#         echo $mp_term_name_underscore

#         # ===========================
#         # Data
#         # ===========================
#         cp -r data/network/mp_term_name/"$mp_term_name_underscore".json.gz test-tsumugi/data/phenotype
        
#         # ===========================
#         # HTML
#         # ===========================
        
#         mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
#         impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/annotation/mptermname_phenotypeurl.tsv)
#         escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

#         # すべての効果量が１の場合、表現型の重症度でのフィルターを削除
#         mode="non-binary-phenotype"
#         grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt && mode="binary_phenotype"

#         if [ "$mode" = "non-binary-phenotype" ] ; then
#             # 効果量のスライダーとSVGカラーバーのHTMLをテンプレートに挿入
#             # -----------------------------
#             # contol-panel.html
#             # -----------------------------
#             template_file="TSUMUGI/template/template-app-html/control-panel.html"
#             insert_file="TSUMUGI/template/template-app-html/control-panel-phenotype.html"
#             output_file="/tmp/control-panel.html"
#             replacement=$(<"$insert_file")
#             # AWKで置換（プレースホルダーを一時的にマーカーに置換し、そこに挿入）
#             awk -v f="$insert_file" 'BEGIN{r=""; while((getline l<f)>0) r=r l ORS; close(f)} {gsub(/XXX_PHENOTYPE/, r); print}' "$template_file" > "$output_file"


#             # -----------------------------
#             # cy-container.html
#             # -----------------------------
#             template_file="TSUMUGI/template/template-app-html/cy-container.html"
#             insert_file="TSUMUGI/template/template-app-html/cy-container-phenotype.html"
#             output_file="/tmp/cy-container.html"
#             replacement=$(<"$insert_file")
#             # AWKで置換（プレースホルダーを一時的にマーカーに置換し、そこに挿入）
#             awk -v f="$insert_file" 'BEGIN{r=""; while((getline l<f)>0) r=r l ORS; close(f)} {gsub(/XXX_PHENOTYPE/, r); print}' "$template_file" > "$output_file"
#         else
#             sed "s|XXX_PHENOTYPE||" TSUMUGI/template/template-app-html/control-panel.html > /tmp/control-panel.html
#             sed "s|XXX_PHENOTYPE||" TSUMUGI/template/template-app-html/cy-container.html > /tmp/cy-container.html
#         fi

#         # -----------------------------
#         # head.html
#         # ----------------------------- 
#         template_file="TSUMUGI/template/template-app-html/head.html"
#         output_file="/tmp/head.html"
        
#         cat "$template_file" |
#             sed "s|XXX_TITLE|${mp_term_name_space}|" |
#             sed "s|XXX_JS_FILE_NAME|${mp_term_name_underscore}|" > "$output_file"

#         # -----------------------------
#         # header.html
#         # ----------------------------- 
#         template_file="TSUMUGI/template/template-app-html/header.html"
#         insert_html="<a href='${impc_url}' target='_blank'>${mp_term_name_space}</a>"
#         output_file="/tmp/header.html"
        
#         cat "$template_file" |
#             sed "s|XXX_TITLE|${insert_html}|" > "$output_file"

#         # -----------------------------
#         # template_app.html
#         # -----------------------------
#         template_file="TSUMUGI/template/template-app-html/template_app.html"
#         head_file="/tmp/head.html"
#         header_file="/tmp/header.html"
#         control_panel_file="/tmp/control-panel.html"
#         cy_container_file="/tmp/cy-container.html"
        
#         escape() {
#                 sed -e 's/[\/&]/\\&/g' -e ':a;N;$!ba;s/\n/\\n/g' "$1"
#         }
#         head_html=$(escape "$head_file")
#         header_html=$(escape "$header_file")
#         control_panel_html=$(escape "$control_panel_file")
#         cy_container_html=$(escape "$cy_container_file")

#         # 置換して \n を改行に戻す
#         sed -e "s|XXX_HEAD|$head_html|" \
#             -e "s|XXX_H1|$header_html|" \
#             -e "s|XXX_CONTROL_PANEL|$control_panel_html|" \
#             -e "s|XXX_CY_CONTAINER|$cy_container_html|" \
#             "$template_file" | sed 's/\\n/\
#         /g' > test-tsumugi/app/phenotype/"$mp_term_name_underscore".html

#         # ===========================
#         # Javascript
#         # ===========================
#         # すべての効果量が１の場合、表現型の重症度でのスライダーを削除

#         if [ "$mode" = "non-binary-phenotype" ] ; then
#             # -----------------------------
#             # filterByNodeColorAndEdgeSize_phenotype.js
#             # -----------------------------
#             cp TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_phenotype.js /tmp/filterByNodeColorAndEdgeSize_phenotype.js
#             # -----------------------------
#             # node_color_slider.js
#             # -----------------------------
#             template_file="TSUMUGI/template/template-app-js/template_app.js"
#             node_color_initialization="TSUMUGI/template/template-app-js/node_color_initialization.js"
#             node_color_update="TSUMUGI/template/template-app-js/node_color_update.js"
#             output_file="/tmp/template_app.js"

#             cat "$template_file" |
#                 sed -e "/XXX_NODE_COLOR_INITIALIZATION/r ${node_color_initialization}" |
#                 sed -e "/XXX_NODE_COLOR_INITIALIZATION/d" |
#                 sed -e "/XXX_NODE_COLOR_UPDATE/r ${node_color_update}" |
#                 sed -e "/XXX_NODE_COLOR_UPDATE/d"  > "$output_file"

#         else
#             # -----------------------------
#             # filterByNodeColorAndEdgeSize_phenotype.js
#             # -----------------------------
#             grep -v "REMOVE_THIS_LINE_IF_BINARY_PHENOTYPE" TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_phenotype.js > /tmp/filterByNodeColorAndEdgeSize_phenotype.js
#             # -----------------------------
#             # node_color_slider.js
#             # -----------------------------
#             grep -v -e "XXX_NODE_COLOR_INITIALIZATION" -e "XXX_NODE_COLOR_UPDATE" TSUMUGI/template/template-app-js/template_app.js > /tmp/template_app.js

#         fi

#         # -----------------------------
#         # XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE
#         # -----------------------------
#         template_file="/tmp/template_app.js"
#         insert_file="/tmp/filterByNodeColorAndEdgeSize_phenotype.js"
#         json_path="loadJSONGz('../../data/phenotype/XXX_NAME.json.gz')"

#         # Perform the replacement using sed with the 'r' command
#         cat "$template_file" |
#         sed -e "/XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE/r ${insert_file}" |
#         sed -e "/XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE/d" |
#         # XXX_ELEMENTSの置換
#         sed "s|XXX_ELEMENTS|${json_path}|g" |
#         # XXX_TARGET_PHENOTYPEの置換
#         sed "s|XXX_TARGET_PHENOTYPE|${mp_term_name_space}|g" |
#         # XXX_NAMEの置換
#         sed "s|XXX_NAME|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
#         # if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
#         #     cat TSUMUGI/template/phenotype/template_network_phenotype.js |
#         #         sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
#         #         grep -v "REMOVE_THIS_LINE" |
#         #         sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
#         # else
#         #     cat TSUMUGI/template/phenotype/template_network_phenotype.js |
#         #         grep -v "REMOVE_THIS_LINE" |
#         #         sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
#         # fi

#     done

## Phenotype

In [12]:
import os
import shutil
from pathlib import Path

# ========== ファイル処理ユーティリティ ==========

def read_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read()

def write_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)

# ========== データ収集系関数 ==========

def get_target_phenotypes(dir_path):
    targets = []
    for file in Path(dir_path).glob("*.csv"):
        name = file.stem
        if name.startswith(("male_infertility", "increased_fasting", "prewe")):
            targets.append(name)
    return targets

def get_impc_url(mp_term_name_space, tsv_path):
    with open(tsv_path, encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if parts[0] == mp_term_name_space:
                return parts[1]
    return ""

def is_binary(mp_term, binary_file_path):
    with open(binary_file_path) as f:
        return any(line.strip() == mp_term for line in f)

# ========== テンプレート系関数 ==========

def replace_placeholder(template, placeholder, insert_text):
    return template.replace(placeholder, insert_text)

def inject_html(template_path, insert_path, placeholder, output_path):
    template = read_file(template_path)
    insert = read_file(insert_path)
    updated = replace_placeholder(template, placeholder, insert)
    write_file(output_path, updated)

def generate_simple_html(template_path, output_path, replacements):
    content = read_file(template_path)
    for key, value in replacements.items():
        content = content.replace(key, value)
    write_file(output_path, content)

# ========== HTML生成系関数 ==========

def generate_all_html(mp_term, mp_term_name_space, impc_url, mode):
    # control-panel と cy-container
    for part in ["control-panel", "cy-container"]:
        template_path = f"TSUMUGI/template/template-app-html/{part}.html"
        output_path = f"/tmp/{part}.html"
        if mode == "non-binary-phenotype":
            insert_path = f"TSUMUGI/template/template-app-html/{part}-phenotype.html"
            inject_html(template_path, insert_path, "XXX_PHENOTYPE", output_path)
        else:
            generate_simple_html(template_path, output_path, {"XXX_PHENOTYPE": ""})

    # head.html
    generate_simple_html(
        "TSUMUGI/template/template-app-html/head.html",
        "/tmp/head.html",
        {
            "XXX_TITLE": mp_term_name_space,
            "XXX_JS_FILE_NAME": mp_term
        }
    )

    # header.html
    header_insert = f"<a href='{impc_url}' target='_blank'>{mp_term_name_space}</a>"
    generate_simple_html(
        "TSUMUGI/template/template-app-html/header.html",
        "/tmp/header.html",
        {"XXX_TITLE": header_insert}
    )

    # template_app.html → 完成版HTML
    template = read_file("TSUMUGI/template/template-app-html/template_app.html")
    final_html = template \
        .replace("XXX_HEAD", read_file("/tmp/head.html")) \
        .replace("XXX_H1", read_file("/tmp/header.html")) \
        .replace("XXX_CONTROL_PANEL", read_file("/tmp/control-panel.html")) \
        .replace("XXX_CY_CONTAINER", read_file("/tmp/cy-container.html"))

    write_file(f"test-tsumugi/app/phenotype/{mp_term}.html", final_html)

# ========== JavaScript生成関数 ==========

def generate_javascript(mp_term, mp_term_name_space, mode):
    template_app_path = "/tmp/template_app.js"

    if mode == "non-binary-phenotype":
        shutil.copy(
            "TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_phenotype.js",
            "/tmp/filterByNodeColorAndEdgeSize_phenotype.js"
        )

        template = read_file("TSUMUGI/template/template-app-js/template_app.js")
        init = read_file("TSUMUGI/template/template-app-js/node_color_initialization.js")
        update = read_file("TSUMUGI/template/template-app-js/node_color_update.js")
        template = template.replace("XXX_NODE_COLOR_INITIALIZATION", init).replace("XXX_NODE_COLOR_UPDATE", update)
        write_file(template_app_path, template)

    else:
        # Binary phenotype の処理
        lines = read_file("TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_phenotype.js").splitlines()
        filtered_lines = "\n".join(line for line in lines if "REMOVE_THIS_LINE_IF_BINARY_PHENOTYPE" not in line)
        write_file("/tmp/filterByNodeColorAndEdgeSize_phenotype.js", filtered_lines)

        template = read_file("TSUMUGI/template/template-app-js/template_app.js")
        template = template.replace("XXX_NODE_COLOR_INITIALIZATION", "").replace("XXX_NODE_COLOR_UPDATE", "")
        write_file(template_app_path, template)

    # 最終JS生成
    main_template = read_file(template_app_path)
    filter_js = read_file("/tmp/filterByNodeColorAndEdgeSize_phenotype.js")

    final_js = main_template \
        .replace("XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE", filter_js) \
        .replace("XXX_ELEMENTS", f"loadJSONGz('../../data/phenotype/{mp_term}.json.gz')") \
        .replace("XXX_TARGET_PHENOTYPE", mp_term_name_space) \
        .replace("XXX_NAME", mp_term)

    write_file(f"test-tsumugi/app/phenotype/{mp_term}.js", final_js)


In [13]:
# ========== メイン処理 ==========
phenotypes = get_target_phenotypes("data/mp_term_name")

for mp_term in phenotypes:
    print(f"Processing: {mp_term}")
    mp_term_name_space = mp_term.replace("_", " ")
    impc_url = get_impc_url(mp_term_name_space, "data/annotation/mptermname_phenotypeurl.tsv")
    mode = "binary_phenotype" if is_binary(mp_term, "data/annotation/binary_phenotypes.txt") else "non-binary-phenotype"

    # データコピー
    shutil.copy(
        f"data/network/mp_term_name/{mp_term}.json.gz",
        f"test-tsumugi/data/phenotype/{mp_term}.json.gz"
    )

    generate_all_html(mp_term, mp_term_name_space, impc_url, mode)
    generate_javascript(mp_term, mp_term_name_space, mode)


Processing: increased_fasting_circulating_glucose_level
Processing: male_infertility
Processing: preweaning_lethality,_complete_penetrance
Processing: preweaning_lethality,_incomplete_penetrance


## Gene Symbol

In [14]:
# %%bash

# ########################################
# # Gene Symbol
# ########################################

# cat data/overlap/available_gene_symbols.txt |
#     grep -e Rab10 -e Asxl1 -e Ddx46 -e Ap3b2 -e Kcnma1 | # <- ここで興味のあるgene symbolを選択
#     while read gene_symbol; do
#         echo $gene_symbol

#         # ===========================
#         # Data
#         # ===========================

#         cp -f data/network/gene_symbol/"$gene_symbol".json.gz test-tsumugi/data/genesymbol

#         mgi_id=$(awk -F "\t" -v key="$gene_symbol" '$1 == key {print $2}' data/annotation/symbol_mgiid.tsv)
#         impc_url="https://www.mousephenotype.org/data/genes/"${mgi_id}
#         escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

#         # ===========================
#         # HTML
#         # ===========================
#         sed "s|XXX_PHENOTYPE||" TSUMUGI/template/template-app-html/control-panel.html > /tmp/control-panel.html
#         sed "s|XXX_PHENOTYPE||" TSUMUGI/template/template-app-html/cy-container.html > /tmp/cy-container.html

#         # -----------------------------
#         # head.html
#         # ----------------------------- 
#         template_file="TSUMUGI/template/template-app-html/head.html"
        
#         cat "$template_file" |
#             sed "s|XXX_TITLE|${gene_symbol}|" |
#             sed "s|XXX_JS_FILE_NAME|${gene_symbol}|" > /tmp/head.html

#         # -----------------------------
#         # header.html
#         # ----------------------------- 
#         template_file="TSUMUGI/template/template-app-html/header.html"
#         insert_html="<a href='${impc_url}' target='_blank'>${gene_symbol}</a>"
        
#         sed "s|XXX_TITLE|${insert_html}|" "$template_file" > /tmp/header.html

#         # -----------------------------
#         # template_app.html
#         # -----------------------------
#         template_file="TSUMUGI/template/template-app-html/template_app.html"
#         head_file="/tmp/head.html"
#         header_file="/tmp/header.html"
#         control_panel_file="/tmp/control-panel.html"
#         cy_container_file="/tmp/cy-container.html"
        
#         escape() {
#                 sed -e 's/[\/&]/\\&/g' -e ':a;N;$!ba;s/\n/\\n/g' "$1"
#         }
#         head_html=$(escape "$head_file")
#         header_html=$(escape "$header_file")
#         control_panel_html=$(escape "$control_panel_file")
#         cy_container_html=$(escape "$cy_container_file")

#         # 置換して \n を改行に戻す
#         sed -e "s|XXX_HEAD|$head_html|" \
#             -e "s|XXX_H1|$header_html|" \
#             -e "s|XXX_CONTROL_PANEL|$control_panel_html|" \
#             -e "s|XXX_CY_CONTAINER|$cy_container_html|" \
#             "$template_file" |sed 's/\\n/\
#                 /g' > test-tsumugi/app/genesymbol/"$gene_symbol".html

#         # ===========================
#         # Javascript
#         # ===========================
#         # cat TSUMUGI/template/template-app-htmlsymbol/template_network_genesymbol.js |
#         #     sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
#         #     grep -v "REMOVE_THIS_LINE" |
#         #     sed "s|XXX_genesymbol|${gene_symbol}|g" > test-tsumugi/app/genesymbol/"$gene_symbol".js


#         # -----------------------------
#         # node_color_slider.js
#         # -----------------------------
#         grep -v -e "XXX_NODE_COLOR_INITIALIZATION" -e "XXX_NODE_COLOR_UPDATE" TSUMUGI/template/template-app-js/template_app.js > /tmp/template_app.js

#         # -----------------------------
#         # XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE
#         # -----------------------------
#         template_file="/tmp/template_app.js"
#         insert_file="TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_genesymbol.js"
#         json_path="loadJSONGz('../../data/genesymbol/XXX_NAME.json.gz')"

#         # Perform the replacement using sed with the 'r' command
#         cat "$template_file" |
#         sed -e "/XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE/r ${insert_file}" |
#         sed -e "/XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE/d" |
#         # XXX_ELEMENTSの置換
#         sed "s|XXX_ELEMENTS|${json_path}|g" |
#         # XXX_TARGET_PHENOTYPEの置換
#         sed "s|XXX_TARGET_PHENOTYPE||" |
#         # XXX_NAMEの置換
#         sed "s|XXX_NAME|${gene_symbol}|g" > test-tsumugi/app/genesymbol/"$gene_symbol".js

#     done

In [15]:
# ========== Gene Symbol用 HTML生成関数 ==========
def generate_gene_html(gene_symbol, impc_url):
    # control-panel と cy-container
    for part in ["control-panel", "cy-container"]:
        template_path = f"TSUMUGI/template/template-app-html/{part}.html"
        output_path = f"/tmp/{part}.html"
        generate_simple_html(template_path, output_path, {"XXX_PHENOTYPE": ""})

    # head.html
    generate_simple_html(
        "TSUMUGI/template/template-app-html/head.html",
        "/tmp/head.html",
        {
            "XXX_TITLE": gene_symbol,
            "XXX_JS_FILE_NAME": gene_symbol
        }
    )

    # header.html
    header_insert = f"<a href='{impc_url}' target='_blank'>{gene_symbol}</a>"
    generate_simple_html(
        "TSUMUGI/template/template-app-html/header.html",
        "/tmp/header.html",
        {"XXX_TITLE": header_insert}
    )

    # template_app.html
    template = read_file("TSUMUGI/template/template-app-html/template_app.html")
    final_html = template \
        .replace("XXX_HEAD", read_file("/tmp/head.html")) \
        .replace("XXX_H1", read_file("/tmp/header.html")) \
        .replace("XXX_CONTROL_PANEL", read_file("/tmp/control-panel.html")) \
        .replace("XXX_CY_CONTAINER", read_file("/tmp/cy-container.html"))

    write_file(f"test-tsumugi/app/genesymbol/{gene_symbol}.html", final_html)

# ========== JavaScript生成関数 ==========
def generate_gene_javascript(gene_symbol):
    # template_app.js のベース作成（特定部分除去）
    template_lines = read_file("TSUMUGI/template/template-app-js/template_app.js").splitlines()
    filtered_lines = [line for line in template_lines if "XXX_NODE_COLOR_INITIALIZATION" not in line and "XXX_NODE_COLOR_UPDATE" not in line]
    write_file("/tmp/template_app.js", "\n".join(filtered_lines))

    # JS中に埋め込むコードを差し込み
    template = read_file("/tmp/template_app.js")
    insert = read_file("TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_genesymbol.js")

    final_js = template.replace("XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE", insert) \
        .replace("XXX_ELEMENTS", "loadJSONGz('../../data/genesymbol/XXX_NAME.json.gz')") \
        .replace("XXX_TARGET_PHENOTYPE", "") \
        .replace("XXX_NAME", gene_symbol)

    write_file(f"test-tsumugi/app/genesymbol/{gene_symbol}.js", final_js)

# ========== MGI ID 取得関数 ==========
def get_mgi_id(gene_symbol, tsv_path="data/annotation/symbol_mgiid.tsv"):
    with open(tsv_path, encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if parts[0] == gene_symbol:
                return parts[1]
    return ""

In [16]:
# ========== メイン処理 ==========
target_genes = ["Rab10", "Asxl1", "Ddx46", "Ap3b2", "Kcnma1"]
with open("data/overlap/available_gene_symbols.txt", encoding='utf-8') as f:
    available = {line.strip() for line in f}

for gene_symbol in target_genes:
    if gene_symbol not in available:
        continue

    print(f"Processing: {gene_symbol}")

    # データコピー
    src = f"data/network/gene_symbol/{gene_symbol}.json.gz"
    dst = f"test-tsumugi/data/genesymbol/{gene_symbol}.json.gz"
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(src, dst)

    # URL生成
    mgi_id = get_mgi_id(gene_symbol)
    impc_url = f"https://www.mousephenotype.org/data/genes/{mgi_id}"

    # HTMLとJS生成
    generate_gene_html(gene_symbol, impc_url)
    generate_gene_javascript(gene_symbol)


Processing: Rab10
Processing: Asxl1
Processing: Ddx46
Processing: Ap3b2
Processing: Kcnma1


## Gene List

In [17]:
# %%bash

# ########################################
# # Gene List
# ########################################

# ##########################################
# # HTML
# ##########################################

# sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/control-panel.html > /tmp/control-panel.html
# sed "s|XXX_PHENOTYPE||" TSUMUGI/template/html/cy-container.html > /tmp/cy-container.html

# # -----------------------------
# # head.html
# # ----------------------------- 
# template_file="TSUMUGI/template/html/head.html"

# cat "$template_file" |
#     sed "s|XXX_TITLE|Gene List|" |
#     sed "s|XXX_JS_FILE_NAME|network_genelist|" > /tmp/head.html

# # -----------------------------
# # header.html
# # ----------------------------- 
# template_file="TSUMUGI/template/html/header.html"

# sed "s|XXX_TITLE|gene list|" "$template_file" > /tmp/header.html

# # -----------------------------
# # template_app.html
# # -----------------------------
# template_file="TSUMUGI/template/html/template_app.html"
# head_file="/tmp/head.html"
# header_file="/tmp/header.html"
# control_panel_file="/tmp/control-panel.html"
# cy_container_file="/tmp/cy-container.html"

# escape() {
#         sed -e 's/[\/&]/\\&/g' -e ':a;N;$!ba;s/\n/\\n/g' "$1"
# }

# head_html=$(escape "$head_file")
# header_html=$(escape "$header_file")
# control_panel_html=$(escape "$control_panel_file")
# cy_container_html=$(escape "$cy_container_file")

# # 置換して \n を改行に戻す
# sed -e "s|XXX_HEAD|$head_html|" \
#     -e "s|XXX_H1|$header_html|" \
#     -e "s|XXX_CONTROL_PANEL|$control_panel_html|" \
#     -e "s|XXX_CY_CONTAINER|$cy_container_html|" \
#     "$template_file" | sed 's/\\n/\
# /g' > test-tsumugi/app/genelist/network_genelist.html

# ##########################################
# # Javascript
# ##########################################
# cat TSUMUGI/template/genelist/template_network_genelist.js |
#     sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
#     grep -v "REMOVE_THIS_LINE" > test-tsumugi/app/genelist/network_genelist.js


## Gene List

In [18]:
# ========== Gene List用 HTML生成関数 ==========
def generate_genelist_html():
    # control-panel と cy-container
    for part in ["control-panel", "cy-container"]:
        template_path = f"TSUMUGI/template/template-app-html/{part}.html"
        output_path = f"/tmp/{part}.html"
        generate_simple_html(template_path, output_path, {"XXX_PHENOTYPE": ""})

    # head.html
    generate_simple_html(
        "TSUMUGI/template/template-app-html/head.html",
        "/tmp/head.html",
        {
            "XXX_TITLE": "Gene List",
            "XXX_JS_FILE_NAME": "network_genelist"
        }
    )

    # header.html
    header_insert = "gene list"
    generate_simple_html(
        "TSUMUGI/template/template-app-html/header.html",
        "/tmp/header.html",
        {"XXX_TITLE": header_insert}
    )

    # template_app.html
    template = read_file("TSUMUGI/template/template-app-html/template_app.html")
    final_html = template \
        .replace("XXX_HEAD", read_file("/tmp/head.html")) \
        .replace("XXX_H1", read_file("/tmp/header.html")) \
        .replace("XXX_CONTROL_PANEL", read_file("/tmp/control-panel.html")) \
        .replace("XXX_CY_CONTAINER", read_file("/tmp/cy-container.html"))

    write_file(f"test-tsumugi/app/genelist/network_genelist.html", final_html)

# ========== JavaScript生成関数 ==========
def generate_genelist_javascript():
    # template_app.js のベース作成（特定部分除去）
    template_lines = read_file("TSUMUGI/template/template-app-js/template_app.js").splitlines()
    filtered_lines = [line for line in template_lines if "XXX_NODE_COLOR_INITIALIZATION" not in line and "XXX_NODE_COLOR_UPDATE" not in line]
    write_file("/tmp/template_app.js", "\n".join(filtered_lines))

    # JS中に埋め込むコードを差し込み
    template = read_file("/tmp/template_app.js")
    insert = read_file("TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_genelist.js")

    final_js = template.replace("XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE", insert) \
        .replace("XXX_ELEMENTS", "JSON.parse(localStorage.getItem('elements'))") \
        .replace("XXX_TARGET_PHENOTYPE", "") \
        .replace("XXX_NAME", "geneList")

    write_file(f"test-tsumugi/app/genelist/network_genelist.js", final_js)


In [19]:
# ========== メイン処理 ==========
# HTMLとJS生成
generate_genelist_html()
generate_genelist_javascript()


In [20]:
%%bash

prettier --write \
    "test-tsumugi/index.html" \
    "test-tsumugi/js/**/*" \
    "test-tsumugi/app/**/*" \
    --print-width 120 --prose-wrap never --tab-width 4 > /dev/null 2>&1



In [21]:
%%bash

uname -a # OS name
date +"%Y/%m/%d %H:%M:%S" # Last update

Linux Sycom-2021 5.15.167.4-microsoft-standard-WSL2 #1 SMP Tue Nov 5 00:21:55 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
2025/04/08 14:12:06
